# Analysis of missing pulses   

For a given flux and exposure time this notebook analyzes the missing ('non-reconstructed') photons and plots the
distribution of separations to their partner ('Bad-reconstructed' pulse).

1. Import modules   
2. Read parameters of simulation   
3. Analysis of bad/non-reconstructed pulses   
   * 3.1. Check distances between a missing photon and its corresponding "bad-reconstructed" photon:    
        - For each simulation:   
            * read CSV file with assignation of *missing* & *bad-reconstructed*   
            * for each *missing* photon: get *bad-reconstructed* partner   
                * read info in `piximpact` file   
                * calculate minimum of the distances to all *bad-reconstructed*: this is its partner   
                * save distance to global list of distances    
                * Alert if:   
                    * No *bad-reconstructed* photon is found for each *missing* photon: raise Error    
                    * Separation [*missing*-*bad_renconstructed*] > 100: raise Error
                    * Separation [*missing*-*bad_renconstructed*] > 30: warning to check particular cases    
    * 3.2 Plot histograms of:   
        - separations   
        - energies of missing photons   
        - energies of badrecons photons   


In [ ]:
from IPython.display import display, Image
display(Image("images/pileup.png", width=350))

### Import modules

In [ ]:
# Do standard imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits, ascii
from astropy.table import Table
from astropy.visualization import hist

import os
import ast
import auxiliary 

### Set simulation parameters

In [ ]:
nsims = 100
fluxDir = f"{os.getcwd()}/flux0.50mcrab/"
exposure = 4331
sampling_rate=130210 #Hz
secondary_samples = 1563
auxiliary.verbose = 1
global_csv_file = "info_nofilt_infoc_global_0.500mCrab.csv"

## Analysis of missing and bad-reconstructed photons

### Pile-up photons separations 

Check distances between a missing photon and its corresponding "bad-reconstructed" photon

In [ ]:
missing_distances = list()
missing_energies  = list()
badrecons_energies = list()
badrecons_energies_secondaries = list()
badrecons_energies_lowres = list()
badrecons_energies_primaries = list()
impact_energies = list()

for i in range(nsims):
    isim = i + 1
    #if isim > 1:
    #    continue
    csv_file = f"{fluxDir}/sim_{isim}/00_info_nofilt_infoc_sim{isim}_missing.csv"
    missing_table = pd.read_csv(csv_file, converters={"Non-reconstructed photons": ast.literal_eval,
                                                      "Bad-reconstructed photons": ast.literal_eval,
                                                      "GRADE1 Bad-recons": ast.literal_eval,
                                                      "GRADE2 Bad-recons": ast.literal_eval})
    
    #read table row by row:
    # Frist column is an integer value that represents the pixel id
    # Second column is a list of integers that represents the PH_ID of the missing photons
    # Third column is a list of integers that represents the PH_ID of the photons that are bad reconstructed    
    for i, row in missing_table.iterrows():
        ipixel = row["Pixel"]
        missing_phs_id = row["Non-reconstructed photons"]
        bad_recons_phs_id = row["Bad-reconstructed photons"]
        bad_recons_grade1 = row["GRADE1 Bad-recons"]
        bad_recons_grade2 = row["GRADE2 Bad-recons"]
        auxiliary.vprint(f"sim {isim}, pixel {ipixel}, missing photons {missing_phs_id}, bad reconstructed photons {bad_recons_phs_id}")
        # identify sirena file
        sirena_file = f"{fluxDir}/sim_{isim}/crab_flux0.50_Emin2_Emax10_exp{exposure}_RA0.0_Dec0.0_nofilt_infoc_pixel{ipixel}_sirena.fits"
        # identify piximpact file for pixel
        piximpact_file = f"{fluxDir}/sim_{isim}/crab_flux0.50_Emin2_Emax10_exp{exposure}_RA0.0_Dec0.0_nofilt_infoc_pixel{ipixel}_piximpact.fits"
        # read TIME and PH_ID columns of piximpact FITS file
        with fits.open(piximpact_file) as hdul:
            piximpact_data = hdul[1].data
            time = piximpact_data["TIME"].copy()
            ph_id = piximpact_data["PH_ID"].copy()
            simenergy = piximpact_data["ENERGY"].copy()
            # add values in simenergy to impact_energies
            impact_energies.extend(simenergy)
        # foreach missing photon, find the minimum TIME distance to the bad reconstructed photons (find its 'partner')
        for imissing in missing_phs_id:
            missing_time = time[ph_id == imissing][0] 
            min_time_diff_samples = float("inf")   
            # find the bad reconstructed photon that is closest in time to the missing photon
            min_bad = None
            for ibad in bad_recons_phs_id:
                bad_time = time[ph_id == ibad][0]
                time_diff_samples = np.abs(missing_time - bad_time)*sampling_rate
                if time_diff_samples < min_time_diff_samples:
                    min_time_diff_samples = time_diff_samples
                    min_bad = ibad
            if min_bad is None:
                print(f"sim {isim}, pixel {ipixel}: no bad ph for missing ph {imissing}")
                raise ValueError("No bad reconstructed photon found for missing photon")
            if min_time_diff_samples > 100:
                print(f"sim {isim}, pixel {ipixel}: missing ph {imissing} and bad ph {min_bad} are separated by {min_time_diff_samples:.2f} samples")
                raise ValueError("Time difference between missing and bad reconstructed photons is too large")
            if min_time_diff_samples > 30:
                print(f"WARNING:sim {isim}, pixel {ipixel}: missing ph {imissing} and bad ph {min_bad} are separated by {min_time_diff_samples:.2f} samples")
    
            # append the minimum time difference to the list of missing distances
            missing_distances.append(min_time_diff_samples)
            missing_energies.append(simenergy[ph_id == imissing][0])
            auxiliary.vprint(f"sim {isim}, pixel {ipixel}, missing ph {imissing}, bad ph {min_bad}, min time diff {min_time_diff_samples:.2f}")
        
        # check bad-recons photons
        for ib in range(len(bad_recons_phs_id)):
            badr = bad_recons_phs_id[ib]
            badr_energy = simenergy[ph_id == badr][0]
            badr_grade1 = bad_recons_grade1[ib] 
            badr_grade2 = bad_recons_grade2[ib]
            if badr_grade2 <= secondary_samples:
                auxiliary.vprint(f"........bad-recons ph {badr}, grade1 {badr_grade1}, grade2 {badr_grade2}: SECONDARY")
                badrecons_energies_secondaries.append(badr_energy)
            elif badr_grade1 == 8:
                auxiliary.vprint(f"........bad-recons ph {badr}, grade1 {badr_grade1}, grade2 {badr_grade2}: LOWRES")
                badrecons_energies_lowres.append(badr_energy)
            else:
                auxiliary.vprint(f"........bad-recons ph {badr}, grade1 {badr_grade1}, grade2 {badr_grade2}: PRIMARY")
                badrecons_energies_primaries.append(badr_energy)
            badrecons_energies.append(badr_energy)


### Distribution of pileup separations and energies

In [ ]:
# read global CSV table with info of all simulations
# look for "Nimpacts" column info selecting where the 'exposure[s]' matches the exposure of the simulations and the 'flux[mcrab]' matches the flux of the simulations
global_table = pd.read_csv(global_csv_file)
global_table = global_table[global_table["exposure[s]"] == exposure]
global_table = global_table[global_table["flux[mcrab]"] == 0.50]
global_table = global_table[global_table["filter"] == "nofilt"]
# get total number of impacts (for all 'simulation')
Nimpacts = global_table["Nimpacts"].sum()
print(f"Total number of impacts: {Nimpacts}")
Nmissing = len(missing_distances)
print(f"Total number of missing impacts: {Nmissing}")
Nbadrecons = len(badrecons_energies)
print(f"Total number of bad reconstructed impacts: {Nbadrecons}")

In [ ]:
# create a figure with two plots
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(12,6))
# In ax1: plot histogram of distances (in samples) for missing photons
#ax1.hist(missing_distances, bins=50, edgecolor='black')
hist(missing_distances, bins='scott', ax=ax1, edgecolor='black')
ax1.set_xlabel("Time difference (samples)")
ax1.set_ylabel("# missing photons")
ax1.set_title("Time separations of missing photons")
# write text on the plot: number of simulations, flux, exposure, sampling rate
text = f"nsims = {nsims}\nNimpacts = {Nimpacts}\nNmissing = {Nmissing}\nflux = 0.50 mCrab\nexposure = {exposure} s\nsampling rate = {sampling_rate} Hz"
ax1.text(0.5, 0.95, text, transform=ax1.transAxes, fontsize=10, verticalalignment='top')

# In ax2:
all_data = [impact_energies, badrecons_energies, missing_energies]
# plot histogram of energies of impact photons
#ax2.hist(impact_energies, bins=50, edgecolor='black', alpha=0.5, label="impact photons")
# plot histogram of energies of missing & bad-reconstructed photons
#ax2.hist(missing_energies, bins=50, alpha=0.5, histtype='step',label=["missing photons"])
hist(missing_energies,ax=ax2, alpha=0.8, bins='scott',histtype='step',label=["missing photons"])
hist(badrecons_energies,ax=ax2,alpha=0.8, bins='scott',histtype='step',label=["bad-recons photons\n(all)"])
hist(badrecons_energies_secondaries,ax=ax2,alpha=0.5, bins='scott',label=["bad-recons photons\n(secondary)"])
hist(badrecons_energies_lowres,ax=ax2,alpha=0.5, bins='scott',label=["bad-recons photons\n(low-res)"])
hist(badrecons_energies_primaries,ax=ax2,alpha=0.5, bins='scott',histtype='step',label=["bad-recons photons\n(primary)"])
# plot histogram of energies of bad-reconstructed photons
ax2.legend()
ax2.set_xlabel("Energy (keV)")
ax2.set_ylabel("# photons")
ax2.set_title("Photon energy distribution")
# write text on the plot: number of simulations, flux, exposure, sampling rate
text = f"nsims = {nsims}\nNimpacts = {Nimpacts}\nNmissing = {Nmissing}\nflux = 0.50 mCrab\nexposure = {exposure} s"
ax2.text(0.5, 0.95, text, transform=ax2.transAxes, fontsize=10, verticalalignment='top')
plt.show()
